시공사 랭킹 변수가 포함됨

In [44]:
import pandas as pd
import numpy as np
from haversine import haversine
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow import keras
import numpy as np
from numpy import concatenate
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
import random

함수정의

In [45]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
      cols.append(df.shift(i))
      names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
      cols.append(df.shift(-i))
      if i == 0:
          names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
      else:
          names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
      agg.dropna(inplace=True)
  return agg

In [46]:
#기준 아파트로부터 'near_range' km 내에 존재하는 아파트 코드 반환
def finding_near_apt(gizun,near_range):
    start = (float(gizun['GPS_Y']),float(gizun['GPS_X']))
    near_apts = []
    for j in df_range.index:
        goal = (float(df_range['GPS_Y'][j]),float(df_range['GPS_X'][j]))
        if haversine(start, goal) < near_range:
            near_apts.append(df_range['apart_code'][j])
    near_df = pd.DataFrame()
    near_df['apart_code'] = near_apts
    return near_df

In [47]:
#사용하지 않을 과거 데이터 제거
def delete_unuse_var(reframed):
    var_count = int(len(reframed.columns)/4)
    reframed.drop(reframed.columns[var_count*2:(var_count*3) - 1], axis=1, inplace=True)
    reframed.drop(reframed.columns[var_count:(var_count*2) - 1], axis=1, inplace=True)
    reframed.drop(reframed.columns[0:var_count - 1], axis=1, inplace=True)
    return reframed

In [48]:
#데이터프레임에서 일정한 컬럼만 추출하고 값 형태로 반환
def get_float_values(df):
    df = df[['YYYYMM','sale_type', 'area_m3', 'floor', 'const_year', 
    'parking','crime_rate', 'building_year', 'market_dist','busstop_dist', 'subway_dist', 
    'kinder_dist', 'hospital_dist', 'ASPI','YYYY', 'MM', 'HAI', 'PIR', 'LIR', 'GDP', 'GDI', 
    'GNI', 'CPI','avg_interest', 'kospi', 'M2', 'house_CCSI', 'unsold','near_element', 
    'near_middle', 'near_high', 'generation','const_rank','prices']]
    values = df.values
    values = values.astype('float32')
    return values

In [ ]:
#코드들로부터 값을 추출하고 합치기
def get_from_codes(codes):
    for j in codes:
        df1 = globals()['df_'+j]
        values = get_float_values(df1)
        reframed = series_to_supervised(values, 3, 1)
        reframed = delete_unuse_var(reframed)
        df_all = pd.concat([reframed,df_all])
    return df_all

파일 읽기

In [49]:
df = pd.read_csv('final6.csv',encoding='cp949')

In [50]:
#결측값 제거
df = df[df['sale_type'].notna()]
df = df[df['parking'].notna()]
df = df[df['HAI'].notna()]
df = df[df['M2'].notna()]
#정성적변수 정량적 변수로 변환
df.loc[df['sale_type'] == '분양', 'sale_type'] = 0
df.loc[df['sale_type'] == '임대+분양', 'sale_type'] = 1
df.loc[df['sale_type'] == '임대', 'sale_type'] = 2
df.loc[df['sale_type'] == '영구임대', 'sale_type'] = 3
df['area_m3']=df['area_m3'].round(-1)
df = df.astype({'prices':'float64'})
df['prices']=df.groupby(['YYYYMM','apart_code','area_m3'])['prices'].transform('mean')
df['apart_code_area'] = df['apart_code'].map(str) +'_' +df['area_m3'].map(int).map(str)
df_a = df
exiting_code = df['apart_code'].unique()
df_vary = df.drop_duplicates(['apart_code'])
df_range = df_vary[['apart_code','GPS_X','GPS_Y']]
many_df = pd.read_csv('서울시아파트(기준).csv',encoding='cp949')
many_df = many_df[many_df['k-아파트코드'].isin(exiting_code)]

실제로 돌아가는 모델

In [51]:
#6분정도 걸리는듯
r2s = []
inv_ys = []
inv_yhats = []
#exiting_code = df['apart_code'].unique()
test_codes = random.sample(list(exiting_code),10)
for i in test_codes:
    gizun_code = i
    gizun = df_range[df_range['apart_code']==gizun_code]
    near_df = finding_near_apt(gizun,2)
    near_df = pd.merge(near_df,many_df,left_on='apart_code',right_on='k-아파트코드',how='left')
    near_df = near_df[['apart_code','k-전체세대수']]
    gizun_count = near_df[near_df['apart_code']==gizun_code].iloc[0][1]
    indexNames = near_df[near_df['k-전체세대수'] < gizun_count].index
    near_df.drop(indexNames , inplace=True)
    df = df_a
    df = pd.merge(near_df,df,on='apart_code')
    df.pop('k-전체세대수')
    df = df.drop_duplicates(subset=['apart_code_area','YYYYMM'])
    df = df.sort_values(by=["apart_code_area", "YYYYMM"], ascending=[True, True])
    df = df[['apart_code_area', 'YYYYMM','sale_type', 'area_m3', 'floor', 'const_year', 
    'parking','crime_rate', 'building_year', 'market_dist','busstop_dist', 'subway_dist', 
    'kinder_dist', 'hospital_dist', 'ASPI','YYYY', 'MM', 'HAI', 'PIR', 'LIR', 'GDP', 'GDI', 
    'GNI', 'CPI','avg_interest', 'kospi', 'M2', 'house_CCSI', 'unsold','near_element', 
    'near_middle', 'near_high', 'generation','const_rank','prices']]
    codes = df['apart_code_area'].unique()
    groups = df.groupby(df.apart_code_area)
    for j in codes:
        globals()['df_'+j] = groups.get_group(j)
    gizun_code_areas = df[df['apart_code_area'].str.contains(gizun_code)]['apart_code_area'].unique()
    for k in gizun_code_areas:
        gizun_code_area = k
        gizun_df = globals()['df_'+gizun_code_area]
        df_all = pd.DataFrame()
        deleted_code = 'df_'+gizun_code_area
        codes = np.delete(codes, np.where(codes == deleted_code))
        #제거가능
        df_all = get_from_codes(codes)
        train_set = df_all[df_all['var1(t)']<202000]
        df_test = gizun_df
        values = get_float_values(df_test)
        reframed = series_to_supervised(values, 3, 1)
        reframed = delete_unuse_var(reframed)
        test_set = reframed[reframed['var1(t)']>=202000]
        df_set = pd.concat([train_set,test_set])
        train_days =  len(train_set)
        values= df_set.values
        scaled = scaler.fit_transform(values)
        values = scaled
        train_days =  len(train_set)
        #테스트 데이터 얼마나 사용할 건지
        train = values[:train_days]
        test = values[train_days:]
        # split into input and outputs
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        history = model.fit(train_X, train_y, epochs=50, batch_size=72, 
                validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # make a prediction
        try:
            yhat = model.predict(test_X)
            test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
            inv_yhat = concatenate((test_X[:, 0:],yhat), axis=1)
            inv_yhat = scaler.inverse_transform(inv_yhat)
            inv_yhat = inv_yhat[:,-1]
            # invert scaling for actual
            test_y = test_y.reshape((len(test_y), 1))
            inv_y = concatenate((test_X[:, 0:],test_y), axis=1)
            inv_y = scaler.inverse_transform(inv_y)
            inv_y = inv_y[:,-1]
            # calculate RMSE
            RMSE = mean_squared_error(inv_y, inv_yhat)**0.5
            r2 = r2_score(inv_y, inv_yhat)
            r2s.append(r2)
            inv_ys.append(inv_y)
            inv_yhats.append(inv_yhat)
            #만약 해당 값을 데이터 프레임에 넣고 싶다면
        except:
            #만약 해당 아파트의 2020년 이후 거래내역이 존재하지 않을 경우 오류 발생
            print('오류 발생') 
    

Epoch 1/50
16/16 - 3s - loss: 0.1284 - val_loss: 0.5368 - 3s/epoch - 179ms/step
Epoch 2/50
16/16 - 0s - loss: 0.0758 - val_loss: 0.4768 - 106ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 0.0602 - val_loss: 0.3721 - 118ms/epoch - 7ms/step
Epoch 4/50
16/16 - 0s - loss: 0.0511 - val_loss: 0.2930 - 123ms/epoch - 8ms/step
Epoch 5/50
16/16 - 0s - loss: 0.0451 - val_loss: 0.2806 - 134ms/epoch - 8ms/step
Epoch 6/50
16/16 - 0s - loss: 0.0387 - val_loss: 0.2728 - 108ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 0.0347 - val_loss: 0.2554 - 123ms/epoch - 8ms/step
Epoch 8/50
16/16 - 0s - loss: 0.0329 - val_loss: 0.2286 - 156ms/epoch - 10ms/step
Epoch 9/50
16/16 - 0s - loss: 0.0316 - val_loss: 0.2094 - 119ms/epoch - 7ms/step
Epoch 10/50
16/16 - 0s - loss: 0.0304 - val_loss: 0.1850 - 113ms/epoch - 7ms/step
Epoch 11/50
16/16 - 0s - loss: 0.0298 - val_loss: 0.1819 - 113ms/epoch - 7ms/step
Epoch 12/50
16/16 - 0s - loss: 0.0291 - val_loss: 0.1534 - 114ms/epoch - 7ms/step
Epoch 13/50
16/16 - 0s - 

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Epoch 1/50
70/70 - 3s - loss: 0.0870 - val_loss: 0.1585 - 3s/epoch - 44ms/step
Epoch 2/50
70/70 - 0s - loss: 0.0514 - val_loss: 0.1561 - 295ms/epoch - 4ms/step
Epoch 3/50
70/70 - 0s - loss: 0.0392 - val_loss: 0.1786 - 276ms/epoch - 4ms/step
Epoch 4/50
70/70 - 0s - loss: 0.0356 - val_loss: 0.1823 - 283ms/epoch - 4ms/step
Epoch 5/50
70/70 - 0s - loss: 0.0338 - val_loss: 0.1815 - 310ms/epoch - 4ms/step
Epoch 6/50
70/70 - 0s - loss: 0.0333 - val_loss: 0.1668 - 289ms/epoch - 4ms/step
Epoch 7/50
70/70 - 0s - loss: 0.0352 - val_loss: 0.1528 - 292ms/epoch - 4ms/step
Epoch 8/50
70/70 - 0s - loss: 0.0356 - val_loss: 0.1425 - 298ms/epoch - 4ms/step
Epoch 9/50
70/70 - 0s - loss: 0.0341 - val_loss: 0.1333 - 317ms/epoch - 5ms/step
Epoch 10/50
70/70 - 0s - loss: 0.0325 - val_loss: 0.1437 - 283ms/epoch - 4ms/step
Epoch 11/50
70/70 - 0s - loss: 0.0302 - val_loss: 0.1492 - 273ms/epoch - 4ms/step
Epoch 12/50
70/70 - 0s - loss: 0.0285 - val_loss: 0.1535 - 283ms/epoch - 4ms/step
Epoch 13/50
70/70 - 0s - lo

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Epoch 1/50
93/93 - 3s - loss: 0.0764 - val_loss: 0.1479 - 3s/epoch - 31ms/step
Epoch 2/50
93/93 - 0s - loss: 0.0723 - val_loss: 0.0402 - 406ms/epoch - 4ms/step
Epoch 3/50
93/93 - 0s - loss: 0.0419 - val_loss: 0.0226 - 394ms/epoch - 4ms/step
Epoch 4/50
93/93 - 0s - loss: 0.0306 - val_loss: 0.0506 - 375ms/epoch - 4ms/step
Epoch 5/50
93/93 - 0s - loss: 0.0279 - val_loss: 0.0301 - 368ms/epoch - 4ms/step
Epoch 6/50
93/93 - 0s - loss: 0.0223 - val_loss: 0.0836 - 382ms/epoch - 4ms/step
Epoch 7/50
93/93 - 0s - loss: 0.0282 - val_loss: 0.0483 - 375ms/epoch - 4ms/step
Epoch 8/50
93/93 - 0s - loss: 0.0228 - val_loss: 0.0474 - 351ms/epoch - 4ms/step
Epoch 9/50
93/93 - 0s - loss: 0.0203 - val_loss: 0.0191 - 349ms/epoch - 4ms/step
Epoch 10/50
93/93 - 0s - loss: 0.0230 - val_loss: 0.0538 - 349ms/epoch - 4ms/step
Epoch 11/50
93/93 - 0s - loss: 0.0240 - val_loss: 0.0501 - 347ms/epoch - 4ms/step
Epoch 12/50
93/93 - 0s - loss: 0.0208 - val_loss: 0.0547 - 344ms/epoch - 4ms/step
Epoch 13/50
93/93 - 0s - lo

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Epoch 1/50
66/66 - 5s - loss: 0.0594 - val_loss: 0.0923 - 5s/epoch - 83ms/step
Epoch 2/50
66/66 - 0s - loss: 0.0335 - val_loss: 0.0153 - 293ms/epoch - 4ms/step
Epoch 3/50
66/66 - 0s - loss: 0.0255 - val_loss: 0.0156 - 415ms/epoch - 6ms/step
Epoch 4/50
66/66 - 1s - loss: 0.0232 - val_loss: 0.0617 - 519ms/epoch - 8ms/step
Epoch 5/50
66/66 - 0s - loss: 0.0287 - val_loss: 0.0275 - 376ms/epoch - 6ms/step
Epoch 6/50
66/66 - 0s - loss: 0.0245 - val_loss: 0.0232 - 448ms/epoch - 7ms/step
Epoch 7/50
66/66 - 0s - loss: 0.0318 - val_loss: 0.0481 - 339ms/epoch - 5ms/step
Epoch 8/50
66/66 - 0s - loss: 0.0212 - val_loss: 0.0344 - 227ms/epoch - 3ms/step
Epoch 9/50
66/66 - 0s - loss: 0.0303 - val_loss: 0.0581 - 257ms/epoch - 4ms/step
Epoch 10/50
66/66 - 0s - loss: 0.0371 - val_loss: 0.0479 - 283ms/epoch - 4ms/step
Epoch 11/50
66/66 - 0s - loss: 0.0198 - val_loss: 0.0275 - 247ms/epoch - 4ms/step
Epoch 12/50
66/66 - 0s - loss: 0.0194 - val_loss: 0.0395 - 239ms/epoch - 4ms/step
Epoch 13/50
66/66 - 0s - lo

In [60]:
import itertools
#모든 모델의 예측값과 실제값을 모아서 2차원리스트를 1차원 리스트로 변환해서 r2값에 넣어버리기
r2_score(list(itertools.chain(*inv_ys)),list(itertools.chain(*inv_yhats)))

0.9812466131797816